In [59]:
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as td
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  1608


In [3]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
mispell_dict = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
                "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not",
                "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would",
                "i'd've": "i would have", "i'll": "i will", "i'll've": "I will have", "i'm": "i am",
                "i've": "I have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
                "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not",
                "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                "so've": "so have", "so's": "so as", "this's": "this is", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                "there'd've": "there would have", "there's": "there is", "here's": "here is",
                "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                "they'll've": "they will have", "they're": "they are", "they've": "they have",
                "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
                "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not",
                "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color',
                'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
                'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize',
                'youtu ': 'youtube ', 'qoura': 'quora', 'sallary': 'salary', 'whta': 'what',
                'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doi': 'do I',
                'thebest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation',
                'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis',
                'etherium': 'ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017',
                '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess',
                "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization',
                'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

puncts = '\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '”': '"', '“': '"', "£": "e",
                 '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta',
                 '∅': '', '³': '3', 'π': 'pi', '\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
for p in puncts:
    punct_mapping[p] = ' %s ' % p

p = re.compile('(\[ math \]).+(\[ / math \])')
p_space = re.compile(r'[^\x20-\x7e]')

In [5]:
def clean_text(text):
    # clean latex maths
    text = p.sub(' [ math ] ', text)
    # clean invisible chars
    text = p_space.sub(r'', text)
    # clean punctuations
    for punct in punct_mapping:
        if punct in text:
            text = text.replace(punct, punct_mapping[punct])
    tokens = []
    for token in text.split():
        # replace contractions & correct misspells
        token = mispell_dict.get(token.lower(), token)
        tokens.append(token)
    return tokens

In [8]:
def prepare_sequences():
    tokenizer = clean_text # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.ReversibleField(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), # we won't be needing the question id, so we pass in None as the field
                 ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                path="data/train.csv", # the root directory where the data lies
                format='csv',
                # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                skip_header=True, 
                fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT)] # Don't supply the label, since this is the test set
    test = torchtext.data.TabularDataset( 
                path="data/test.csv",
                format="csv",
                skip_header=True,
                fields=test_datafields
    )
    return TEXT, LABEL, train, test

In [9]:
TEXT, LABEL, train, test = prepare_sequences()
TEXT.build_vocab(train, vectors="glove.6B.50d")

In [10]:
train_iter = BucketIterator(
 train, # we pass in the datasets we want the iterator to draw data from
 batch_size=64, 
 sort_key=lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False, # sorting would add bias
 repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)
test_iter = Iterator(
  test,
  batch_size=64,
  sort=False,
  sort_within_batch=False,
  repeat=False)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [79]:
class BiLSTMBaseline(nn.Module):
    def __init__(self, hidden_dim, emb_dim, num_linear):
        super().__init__() # gives us access to nn.Module methods and attributes
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim) # similar to word2vec model
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=1)
        self.linear_layers = []
        for _ in range(num_linear - 1):
            self.linear_layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.linear_layers = nn.ModuleList(self.linear_layers)
        self.predictor = nn.Linear(hidden_dim, 1)
        
    def forward(self, seq):
        out, _ = self.encoder(self.embedding(seq))
        feature = out[-1, :, :]
        for layer in self.linear_layers:
            feature = layer(feature)
            preds = self.predictor(feature)
        return nn.Sigmoid()(preds)

In [80]:
nh = 500
em_sz = 50
nl = 3
num_epochs = 5

In [81]:
lstm = BiLSTMBaseline(nh, em_sz, nl)
print(lstm)

BiLSTMBaseline(
  (embedding): Embedding(173174, 50)
  (encoder): LSTM(50, 500)
  (linear_layers): ModuleList(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=500, bias=True)
  )
  (predictor): Linear(in_features=500, out_features=1, bias=True)
)


In [77]:
lstm
TEXT.vocab.freqs

Counter({'hardy': 40,
         'immodest': 5,
         'onetrust': 1,
         'jugs': 8,
         'haggadah': 2,
         'wilx': 1,
         'fcfs': 1,
         'jaramogi': 1,
         'dabb': 1,
         'marketo': 4,
         'bolshoi': 3,
         'shelrake': 1,
         'antati': 1,
         '1821': 3,
         'erpis': 1,
         'sadomasochism': 3,
         'sanctuarious': 1,
         'criminally': 28,
         'harlequinn': 1,
         'heling': 2,
         'extractional': 1,
         'blackpowder': 2,
         'ps5': 3,
         'asexuality': 10,
         'orton': 6,
         'paradoxical': 13,
         'concussions': 11,
         'youbike': 1,
         'cosplay': 43,
         'melkor': 10,
         'greg': 36,
         'probabilistic': 20,
         'sliver': 5,
         'linseed': 6,
         'monocotyledon': 4,
         'decrese': 3,
         'ofx': 2,
         'bamnoli': 1,
         'guestofaguest': 1,
         'filthiest': 13,
         'eastlake': 3,
         'rmlnlu': 6

In [78]:
criterion = nn.BCELoss()
optimizer = optim.SGD(lstm.parameters(), lr = 0.01, momentum=0.9)
errors = []
for epoch in range(num_epochs):
    print("Epoch={}".format(epoch))
    batch_error = 0
    for data in train_iter:
        optimizer.zero_grad()
        predicted = lstm.forward(data.question_text)
        loss = criterion(predicted.squeeze(), data.target.float())
        batch_error += loss
        loss.backward()
        optimizer.step()
    print("Batch error={}".format(batch_error))
    errors.append(batch_error)

Epoch=0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 